#            Projet DSP 
    Réalisé par : Megder Mohamed Al Amine / Matrache Souhail / Hammoudi Alae

Note : Les codes ci-dessous sont modifiables directement en faisant glisser les boutons en haut à gauche de chaque schéma vers la droite pour augmenter / vers la gauche pour diminuer la valeur.

# Convolution 

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
%matplotlib widget

class DiscreteSignalAnalyzer:
    def __init__(self):
        self.fig, self.axes = plt.subplots(1, 3, figsize=(15, 5))
        plt.close()  # Prevent duplicate display
        
    def analyze_signals(self, retard_delta, decalage_u, debut_n, fin_n):
        n = np.arange(debut_n, fin_n)
        
        # Clear previous plots
        for ax in self.axes:
            ax.clear()
            ax.grid(True)
            ax.set_xlabel('n')
            ax.set_ylabel('Amplitude')
        
        # Plot step function
        self.axes[0].stem(n, np.heaviside(n - decalage_u, 1))
        self.axes[0].set_title(f'Échelon u[n-{decalage_u}]')
        
        # Plot impulse function
        delta = np.array([1 if i == retard_delta else 0 for i in n])
        self.axes[1].stem(n, delta)
        self.axes[1].set_title(f'Impulsion δ[n-{retard_delta}]')
        
        # Plot convolution
        self.axes[2].stem(n, np.heaviside(n - (retard_delta + decalage_u), 1))
        self.axes[2].set_title('Convolution')
        
        plt.tight_layout()
        display(self.fig)

def create_interactive_display():
    analyzer = DiscreteSignalAnalyzer()
    
    interact(analyzer.analyze_signals,
            retard_delta=widgets.IntSlider(
                min=-10, max=10, step=1, value=2,
                description='Retard δ[n]',
                continuous_update=True,
                style={'description_width': 'initial'}
            ),
            decalage_u=widgets.IntSlider(
                min=-10, max=10, step=1, value=0,
                description='Décalage u[n]',
                continuous_update=True,
                style={'description_width': 'initial'}
            ),
            debut_n=widgets.IntSlider(
                min=-20, max=0, step=1, value=-10,
                description='Début',
                style={'description_width': 'initial'}
            ),
            fin_n=widgets.IntSlider(
                min=1, max=20, step=1, value=10,
                description='Fin',
                style={'description_width': 'initial'}
            ))

# Launch the interactive display
create_interactive_display()


interactive(children=(IntSlider(value=2, description='Retard δ[n]', max=10, min=-10, style=SliderStyle(descrip…

par exemple : u[n] * δ[n-2]

1- FFT Spectrum ( Bilateral spectrum of the signal )

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

class SignalAnalyzer:
    def __init__(self):
        self.fs = 80000  # Sampling frequency (Hz)
        self.t = np.linspace(0, 0.01, int(self.fs * 0.01), endpoint=False)
        self.fig, self.ax = plt.subplots(figsize=(12, 6))
        plt.close()

    def analyze_signal(self, A1, f1, A2, f2, A3, f3):
        # Convert kHz to Hz for calculations
        f1_hz = f1 * 1000
        f2_hz = f2 * 1000
        f3_hz = f3 * 1000
        
        # Generate signal with three components
        x_t = (A1 * np.cos(2 * np.pi * f1_hz * self.t) +
               A2 * np.cos(2 * np.pi * f2_hz * self.t) +
               A3 * np.cos(2 * np.pi * f3_hz * self.t))
        
        # FFT computation
        X_f = np.fft.fft(x_t)
        freqs = np.fft.fftfreq(len(self.t), 1/self.fs)
        X_f = X_f / (len(self.t)/2)

        # Clear previous plot
        self.ax.clear()

        # Plot frequency spectrum
        self.ax.stem(freqs/1000, np.abs(X_f), basefmt=" ")
        self.ax.set_xlim(-20, 20)
        self.ax.set_title("Spectre fréquentiel bilatéral")
        self.ax.set_xlabel("Fréquence (kHz)")
        self.ax.set_ylabel("Amplitude")
        self.ax.grid(True)

        plt.tight_layout()
        display(self.fig)

def create_interactive_display():
    analyzer = SignalAnalyzer()
    
    interact(analyzer.analyze_signal,
            A1=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=5,
                description='Amplitude 1',
                style={'description_width': 'initial'}
            ),
            f1=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=2,
                description='Fréquence 1 (kHz)',
                style={'description_width': 'initial'}
            ),
            A2=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=3,
                description='Amplitude 2',
                style={'description_width': 'initial'}
            ),
            f2=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=3,
                description='Fréquence 2 (kHz)',
                style={'description_width': 'initial'}
            ),
            A3=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=4,
                description='Amplitude 3',
                style={'description_width': 'initial'}
            ),
            f3=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=5,
                description='Fréquence 3 (kHz)',
                style={'description_width': 'initial'}
            ))

# Launch the interactive display
create_interactive_display()


interactive(children=(FloatSlider(value=5.0, description='Amplitude 1', max=10.0, step=0.5, style=SliderStyle(…

ici par exemple on choisit la fonction suivante : x(t) = 5 cos(2π 2000 t) + 3 cos(2π 3000 t) + 4 cos(2π 5000 t)

2- Bilateral sampled signal spectrum :

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

class SignalAnalyzer:
    def __init__(self):
        self.fig, self.ax = plt.subplots(figsize=(12, 6))
        plt.close()

    def analyze_signal(self, fs_khz, A1, f1, A2, f2, A3, f3):
        fs = fs_khz * 1000
        f = np.linspace(-20000, 20000, 20000)
        
        # Convert kHz to Hz
        f1_hz = f1 * 1000
        f2_hz = f2 * 1000
        f3_hz = f3 * 1000
        
        def cosine_spectrum(A, f0, f):
            spectrum = np.zeros_like(f)
            spectrum += A * (np.abs(f - f0) < 1)  # Full amplitude for positive frequency
            spectrum += A * (np.abs(f + f0) < 1)  # Full amplitude for negative frequency
            return spectrum
        
        def sampled_cosine_spectrum(A, f0, fs, f):
            spectrum = np.zeros_like(f)
            for k in range(-5, 6):
                spectrum += cosine_spectrum(A, f0 + k*fs, f)
            return spectrum
        
        sampled_spectrum = (sampled_cosine_spectrum(A1, f1_hz, fs, f) + 
                          sampled_cosine_spectrum(A2, f2_hz, fs, f) +
                          sampled_cosine_spectrum(A3, f3_hz, fs, f))

        self.ax.clear()
        self.ax.plot(f/1000, sampled_spectrum)
        self.ax.set_xlim(-20, 20)
        self.ax.set_title("Spectre bilatéral du signal échantillonné")
        self.ax.set_xlabel("Fréquence (kHz)")
        self.ax.set_ylabel("Amplitude")
        self.ax.grid(True)
        plt.tight_layout()
        display(self.fig)

def create_interactive_display():
    analyzer = SignalAnalyzer()
    
    interact(analyzer.analyze_signal,
            fs_khz=widgets.FloatSlider(
                min=0, max=15, step=0.5, value=8,
                description='Fs (kHz)',
                style={'description_width': 'initial'}
            ),
            A1=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=5,
                description='Amplitude 1',
                style={'description_width': 'initial'}
            ),
            f1=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=2,
                description='Fréquence 1 (kHz)',
                style={'description_width': 'initial'}
            ),
            A2=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=3,
                description='Amplitude 2',
                style={'description_width': 'initial'}
            ),
            f2=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=3,
                description='Fréquence 2 (kHz)',
                style={'description_width': 'initial'}
            ),
            A3=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=4,
                description='Amplitude 3',
                style={'description_width': 'initial'}
            ),
            f3=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=5,
                description='Fréquence 3 (kHz)',
                style={'description_width': 'initial'}
            ))

# Launch the interactive display
create_interactive_display()


interactive(children=(FloatSlider(value=8.0, description='Fs (kHz)', max=15.0, step=0.5, style=SliderStyle(des…

3- Bilateral spectrum after ideal LPF :

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

class SignalAnalyzer:
    def __init__(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(12, 10))
        plt.close()

    def analyze_signal(self, fs_khz, fc_khz, A1, f1, A2, f2, A3, f3):
        fs = fs_khz * 1000
        fc = fc_khz * 1000
        f = np.linspace(-20000, 20000, 20000)
        
        # Convert kHz to Hz
        f1_hz = f1 * 1000
        f2_hz = f2 * 1000
        f3_hz = f3 * 1000
        
        def cosine_spectrum(A, f0, f):
            spectrum = np.zeros_like(f)
            spectrum += A * (np.abs(f - f0) < 1)
            spectrum += A * (np.abs(f + f0) < 1)
            return spectrum
        
        def sampled_cosine_spectrum(A, f0, fs, f):
            spectrum = np.zeros_like(f)
            for k in range(-5, 6):
                spectrum += cosine_spectrum(A, f0 + k*fs, f)
            return spectrum
        
        # Original sampled spectrum
        sampled_spectrum = (sampled_cosine_spectrum(A1, f1_hz, fs, f) + 
                          sampled_cosine_spectrum(A2, f2_hz, fs, f) +
                          sampled_cosine_spectrum(A3, f3_hz, fs, f))
        
        # Apply ideal LPF
        filtered_spectrum = sampled_spectrum * (np.abs(f) <= fc)

        # Plot original sampled spectrum
        self.ax1.clear()
        self.ax1.plot(f/1000, sampled_spectrum)
        self.ax1.set_xlim(-20, 20)
        self.ax1.set_title("Spectre bilatéral du signal échantillonné")
        self.ax1.set_xlabel("Fréquence (kHz)")
        self.ax1.set_ylabel("Amplitude")
        self.ax1.grid(True)

        # Plot filtered spectrum
        self.ax2.clear()
        self.ax2.plot(f/1000, filtered_spectrum)
        self.ax2.set_xlim(-20, 20)
        self.ax2.set_title(f"Spectre après filtrage passe-bas (fc = {fc_khz} kHz)")
        self.ax2.set_xlabel("Fréquence (kHz)")
        self.ax2.set_ylabel("Amplitude")
        self.ax2.grid(True)

        plt.tight_layout()
        display(self.fig)

def create_interactive_display():
    analyzer = SignalAnalyzer()
    
    interact(analyzer.analyze_signal,
            fs_khz=widgets.FloatSlider(
                min=0, max=15, step=0.5, value=8,
                description='Fs (kHz)',
                style={'description_width': 'initial'}
            ),
            fc_khz=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=4,
                description='fc (kHz)',
                style={'description_width': 'initial'}
            ),
            A1=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=5,
                description='Amplitude 1',
                style={'description_width': 'initial'}
            ),
            f1=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=2,
                description='Fréquence 1 (kHz)',
                style={'description_width': 'initial'}
            ),
            A2=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=3,
                description='Amplitude 2',
                style={'description_width': 'initial'}
            ),
            f2=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=3,
                description='Fréquence 2 (kHz)',
                style={'description_width': 'initial'}
            ),
            A3=widgets.FloatSlider(
                min=0, max=10, step=0.5, value=4,
                description='Amplitude 3',
                style={'description_width': 'initial'}
            ),
            f3=widgets.FloatSlider(
                min=0, max=10, step=0.1, value=5,
                description='Fréquence 3 (kHz)',
                style={'description_width': 'initial'}
            ))

# Launch the interactive display
create_interactive_display()


interactive(children=(FloatSlider(value=8.0, description='Fs (kHz)', max=15.0, step=0.5, style=SliderStyle(des…

Pour le même exemple : on remarque qu'on obtient une mauvaise restitution du signal après l'application du filtre passe-bas idéal LPF ( f_c = 4KHz) sur le spectre bilatéral du signal échantillonné: 
x(t)_transmission ≠ x(t)_reception. 
Puisque le théorème de Shannon vérifié : 
f_s >= 2 x f_max ---->
8kHz >= 2 x 5kHz ---->
8kHz >= 10kHz (non vérifié) . Par conséquent, il y a apparition du phenommène d'aliasing.

# Bonus
Les filtres nummériques FIR et IIR.

1- FIR

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# FIR Filter Design
class FIRFilter:
    def __init__(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(12, 8))
        
    def design_filter(self, numtaps, cutoff_freq):
        # Normalized cutoff frequency (1.0 corresponds to Nyquist frequency)
        nyq_freq = cutoff_freq / 2
        
        # Design FIR filter
        fir_coeff = signal.firwin(numtaps, nyq_freq, window='hamming')
        
        # Frequency response
        w, h = signal.freqz(fir_coeff)
        
        # Plot impulse response
        self.ax1.stem(fir_coeff)
        self.ax1.set_title('Réponse impulsionnelle du filtre FIR')
        self.ax1.set_xlabel('n (échantillons)')
        self.ax1.grid(True)
        
        # Plot frequency response
        self.ax2.plot(w/np.pi, 20 * np.log10(np.abs(h)))
        self.ax2.set_title('Réponse fréquentielle du filtre FIR')
        self.ax2.set_xlabel('Fréquence normalisée (×π rad/échantillon)')
        self.ax2.set_ylabel('Magnitude (dB)')
        self.ax2.grid(True)
        self.ax2.set_ylim(-100, 5)
        
        plt.tight_layout()
        plt.show()


from ipywidgets import interact, widgets

fir = FIRFilter()
interact(fir.design_filter,
        numtaps=widgets.IntSlider(min=3, max=101, step=2, value=31, description='Ordre'),
        cutoff_freq=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, description='Fréq. de coupure'))



interactive(children=(IntSlider(value=31, description='Ordre', max=101, min=3, step=2), FloatSlider(value=0.5,…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

2- IIR 

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# IIR Filter Design
class IIRFilter:
    def __init__(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(12, 8))
        
    def design_filter(self, order, cutoff_freq, filter_type='butter'):
        # Normalized cutoff frequency
        nyq_freq = cutoff_freq / 2
        
        # Design IIR filter
        if filter_type == 'butter':
            b, a = signal.butter(order, nyq_freq)
        elif filter_type == 'cheby1':
            b, a = signal.cheby1(order, 1, nyq_freq)
        
        # Get frequency response
        w, h = signal.freqz(b, a)
        
        # Plot pole-zero diagram
        z, p, k = signal.tf2zpk(b, a)
        self.ax1.plot(np.real(z), np.imag(z), 'o', label='Zéros')
        self.ax1.plot(np.real(p), np.imag(p), 'x', label='Pôles')
        self.ax1.set_title('Diagramme pôles-zéros')
        self.ax1.grid(True)
        self.ax1.legend()
        
        # Plot frequency response
        self.ax2.plot(w/np.pi, 20 * np.log10(np.abs(h)))
        self.ax2.set_title('Réponse fréquentielle du filtre IIR')
        self.ax2.set_xlabel('Fréquence normalisée (×π rad/échantillon)')
        self.ax2.set_ylabel('Magnitude (dB)')
        self.ax2.grid(True)
        self.ax2.set_ylim(-100, 5)
        
        plt.tight_layout()
        plt.show()
 
from ipywidgets import interact, widgets

iir = IIRFilter()
interact(iir.design_filter,
        order=widgets.IntSlider(min=1, max=10, step=1, value=8, description='Ordre'),
        cutoff_freq=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, description='Fréq. de coupure'),
        filter_type=widgets.Dropdown(options=['butter', 'cheby1'], value='cheby1', description='Type'))


interactive(children=(IntSlider(value=8, description='Ordre', max=10, min=1), FloatSlider(value=0.5, descripti…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>